In [6]:
%load_ext autoreload
%autoreload 2

from pymc_marketing.mmm import MMM, GeometricAdstock, LogisticSaturation
from mmm_eval import PyMCConfig, run_evaluation

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load your data

In [2]:
# Load your data - we have a revenue column and a response column (quantity)
data = pd.read_csv("data/example_data.csv")
data.head()

,date_week,quantity,price,revenue,radio,TV,event_1,event_2,dayofyear
0,2018-04-02,1250.138126,5.000322,6251.093528,31.858002,0.0,0.0,0.0,92
1,2018-04-09,1254.595052,5.015090,6291.906837,11.238848,0.0,0.0,0.0,99
2,2018-04-16,1300.522776,5.029466,6540.934884,29.240027,0.0,0.0,0.0,106
3,2018-04-23,1279.793864,5.043116,6454.149541,7.139855,0.0,0.0,0.0,113
4,2018-04-30,1242.244560,5.055754,6280.483221,38.674515,0.0,0.0,0.0,120


# Define a pymc MMM object

A meridian model requires a few arguments - mostly column mapping

As an example, we also define some adstock and saturation priors for all media channels (based on a Lognormal(0.2, 0.9) distribution).

In [3]:
mmm = MMM(
    date_column="date_week" ,
    channel_columns=["TV","radio"],
    adstock=GeometricAdstock(l_max=4),
    saturation=LogisticSaturation()
)
mmm

# Create an `mmm-eval` config 

The `mmm-eval` package can work with a variety of frameworks. It does this by allowing you to create a config from some simple arguments. These slightly differ between frameworks, but for pymc this involves just

1. the pymc model object
2. the revenue column

Other attributes are defined already in the pymc model object

In [8]:
# Define the fit kwargs if desired
fit_kwargs = { 
    "chains": 4,
    "target_accept": 0.85,
}

# Create the full `mmm-eval` pymc config
config = PyMCConfig.from_model_object(mmm, 
                                      fit_kwargs=fit_kwargs, 
                                      response_column="quantity", 
                                      revenue_column="revenue"
                                      )

# Save this for later if you want to run from CLI
config.save_model_object_to_json(save_path="data/", file_name="saved_config")

PyMCConfig(revenue_column='revenue', response_column='quantity', pymc_model_config=PyMCModelSchema(date_column='date_week', channel_columns=['TV', 'radio'], adstock=GeometricAdstock(prefix='adstock', l_max=4, normalize=True, mode='After', priors={'alpha': Prior("Beta", alpha=1, beta=3, dims="channel")}), saturation=LogisticSaturation(prefix='saturation', priors={'lam': Prior("Gamma", alpha=3, beta=1, dims="channel"), 'beta': Prior("HalfNormal", sigma=2, dims="channel")}), time_varying_intercept=False, time_varying_media=False, sampler_config={}, validate_data=True, control_columns=None, yearly_seasonality=None, adstock_first=True, dag=None, treatment_nodes=None, outcome_node=None), fit_config=PyMCFitSchema(draws=None, tune=None, chains=4, target_accept=0.85, random_seed=None, progressbar=False, return_inferencedata=True), date_column='date_week', channel_columns=['TV', 'radio'], control_columns=None)

# Run the evaluation suite

The evaluation suite requires 3 arguments
1. The framework name
2. The `mmm-eval` version of the config for that framework
3. The dataframe to run the tests on

We can also include the tests to run. If they are not included, it runs all tests by default

In [7]:
# Run the evaluation suite!
results = run_evaluation(framework="pymc_marketing", config=config, data=data)
results

2025-08-02 15:11:22,284 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: holdout_accuracy
2025-08-02 15:11:22,284 - mmm_eval.core.base_validation_test - INFO - Splitting data into train and test sets for holdout_accuracy test
2025-08-02 15:11:28,398 - mmm_eval.core.validation_tests - INFO - Saving the test results for holdout_accuracy test
2025-08-02 15:11:28,399 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: in_sample_accuracy
2025-08-02 15:11:34,380 - mmm_eval.core.validation_tests - INFO - Saving the test results for in_sample_accuracy test
2025-08-02 15:11:34,380 - mmm_eval.core.validation_test_orchestrator - INFO - Running test: cross_validation
2025-08-02 15:11:34,381 - mmm_eval.core.base_validation_test - INFO - Splitting data into train and test sets for cross_validation test
2025-08-02 15:11:34,383 - mmm_eval.core.validation_tests - INFO - Running cross-validation fold 1 of 5
2025-08-02 15:11:40,671 - mmm_eval.core.validation_tests - INF

,general_metric_name,specific_metric_name,metric_value,metric_pass,test_name,timestamp
0,mape,mape,15.736717,False,holdout_accuracy,2025-08-02T15:11:28.399109
1,smape,smape,14.438489,True,holdout_accuracy,2025-08-02T15:11:28.399109
2,r_squared,r_squared,-10.877013,False,holdout_accuracy,2025-08-02T15:11:28.399109
3,mape,mape,6.840552,True,in_sample_accuracy,2025-08-02T15:11:34.380759
4,smape,smape,6.759995,True,in_sample_accuracy,2025-08-02T15:11:34.380759
5,r_squared,r_squared,0.402411,False,in_sample_accuracy,2025-08-02T15:11:34.380759
6,mean_mape,mean_mape,8.450682,True,cross_validation,2025-08-02T15:12:04.090041
7,std_mape,std_mape,5.821450,False,cross_validation,2025-08-02T15:12:04.090041
8,mean_smape,mean_smape,7.947130,True,cross_validation,2025-08-02T15:12:04.090041
9,std_smape,std_smape,5.236134,False,cross_validation,2025-08-02T15:12:04.090041
